<h5> Let us start by importing required packages </h5>

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


<h5> Now let us find the coordinates of Bangalore using geopy package </h5>

In [2]:
address = 'Bangalore,KA'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


<h5> Now we import dataset containing all neighbourhoods of bangalore and their co-ordinates </h5>

In [3]:
path='C:\\Users\\Aniruddh\\Documents\\Bangalore_neighbourhoods.csv'
loc_df=pd.read_csv(path)
loc_df.head()

,Unnamed: 0,Neighborhood,Latitude,Longitude
0,0,Agram,45.813177,15.977048
1,1,Amruthahalli,13.066513,77.596624
2,2,Attur,11.663711,78.533551
3,3,Banaswadi,13.014162,77.651854
4,4,Bellandur,58.235358,26.683116


We drop unrequired column 'Unnamed:0'

In [4]:
loc_df.drop(columns='Unnamed: 0',inplace=True)

<h5> Dataset is saved in a dataframe </h5>

In [5]:
loc_df.head()

,Neighborhood,Latitude,Longitude
0,Agram,45.813177,15.977048
1,Amruthahalli,13.066513,77.596624
2,Attur,11.663711,78.533551
3,Banaswadi,13.014162,77.651854
4,Bellandur,58.235358,26.683116


<h5> Above dataset contains a few areas outside of Bangalore as well.By imposing co-ordinate constraints,let us concentrate on areas of Urban Bangalore </h5>

In [6]:
floc_df=loc_df[(loc_df['Latitude']>12.5266)&(loc_df['Latitude']<13.2847)]
afloc_df=floc_df[(floc_df['Longitude']>77.45)&(floc_df['Longitude']<77.75)]
afloc_df.reset_index(drop=True,inplace=True)
afloc_df


,Neighborhood,Latitude,Longitude
0,Amruthahalli,13.066513,77.596624
1,Banaswadi,13.014162,77.651854
2,Bhattarahalli,13.025800,77.714279
3,Byatarayanapura,13.062074,77.596392
4,Devasandra,12.757227,77.647280
5,Doddanekkundi,12.975720,77.694042
6,Domlur,12.962467,77.638196
7,EPIP,12.977673,77.722200
8,Hoodi,12.991903,77.716201
9,Horamavu,13.027331,77.660151


<h5> Visualizing neighbourhoods of above dataframe </h5>

In [7]:

map_bng = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, neighborhood in zip(afloc_df['Latitude'],afloc_df['Longitude'],afloc_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bng)  
    
map_bng

<h5> To use the Foursquare API,access credentials must be saved first </h5>

In [8]:

CLIENT_ID = 'IFSPW0F2P2WGUQABI32ENUXZ3BMO1S3TRC2QBI3ZTHLS4JVF'
CLIENT_SECRET = 'EWM0ZFTOFICURRX1AOVMQHESZVVHV1UCI1O1WU5GMI3XRCQS'
VERSION = '20180605'

print('Your credentials:')
print('CLIENT_ID:Secure')
print('CLIENT_SECRET:Secure')

Your credentials:
CLIENT_ID:Secure
CLIENT_SECRET:Secure


<h5> Creating a function to find all popular venues(max 100) within a 1 km radius of co-ordinates of each area in a dataframe </h5>

In [9]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h5> Passing details of each neighbourhood in saved dataframe to above function </h5>

In [10]:
bgr_venues = getNearbyVenues(names=afloc_df['Neighborhood'],
                                   latitudes=afloc_df['Latitude'],
                                   longitudes=afloc_df['Longitude']
                                  )

Amruthahalli
Banaswadi
Bhattarahalli
Byatarayanapura
Devasandra
Doddanekkundi
Domlur
EPIP
Hoodi
Horamavu
Indiranagar S.O (Bangalore)
Jakkur
Kannamangala
Kodigehalli
Kundalahalli
Lingarajapuram
Panathur
Rajanakunte
Sadashivanagar
Singanayakanahalli
Vimanapura
Virgonagar
Yelahanka
Adugodi
Bannerghatta
Chickpet
Chikkalasandra
Deepanjalinagar
Doddakallasandra
Girinagar S.O (Bangalore)
Gottigere
Haragadde
Hennagara
Hulimangala
Huskur
Jayanagar H.O
Jigani
Kalkere
Kallubalu
Kathriguppe
Kengeri
Konanakunte
Koramangala
Mallathahalli
Mavalli
Nayandahalli
Ragihalli
Sakalavara
Singasandra
Subramanyapura
Taralu
Thalaghattapura
Yelachenahalli
Achitnagar
Bagalgunte
Basaveshwaranagar
Byatha
Hessarghatta
Jalahalli H.O
Kakolu
Kamakshipalya
Kodigehalli
Laggere
Mathikere
Msrit
Nagarbhavi
Nagasandra S.O (Bangalore)
Peenya Dasarahalli
Shivakote
Tarabanahalli
Vidyaranyapura
Vijayanagar S.O (Bangalore)
Alahalli
Anekal
Anneshwara
Aradeshanahalli
Arakere
Bagalur S.O (Bangalore)
Bettakote
Budigere
Byagadadenahal

<h5> Let us view some of the venues and the details on them returned </h5>

In [11]:
bgr_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amruthahalli,13.066513,77.596624,The Druid Garden,13.063946,77.591492,Brewery
1,Amruthahalli,13.066513,77.596624,Big Straw,13.063414,77.591192,Bubble Tea Shop
2,Amruthahalli,13.066513,77.596624,Swensen's,13.063476,77.590793,Ice Cream Shop
3,Amruthahalli,13.066513,77.596624,Shivas Kabab Corner,13.062748,77.591789,Indian Restaurant
4,Amruthahalli,13.066513,77.596624,Sanjay Dhaba,13.058612,77.593767,Indian Restaurant


In [12]:
bgr_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Achitnagar,4,4,4,4,4,4
Adugodi,45,45,45,45,45,45
Alahalli,1,1,1,1,1,1
Amruthahalli,16,16,16,16,16,16
Anekal,4,4,4,4,4,4
Bagalgunte,4,4,4,4,4,4
Bagalur S.O (Bangalore),2,2,2,2,2,2
Banaswadi,14,14,14,14,14,14
Basaveshwaranagar,17,17,17,17,17,17


In [13]:
print('There are {} uniques categories.'.format(len(bgr_venues['Venue Category'].unique())))

There are 177 uniques categories.


<h5>One-hot encoding is done to get statistical information on venues </h5>

In [14]:
bgr_onehot = pd.get_dummies(bgr_venues[['Venue Category']], prefix="", prefix_sep="")
bgr_onehot['Neighbourhood'] = bgr_venues['Neighborhood'] 
fixed_col = [bgr_onehot.columns[-1]] + list(bgr_onehot.columns[:-1])
bgr_onehot = bgr_onehot[fixed_col]

bgr_onehot.head()

,Neighbourhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Henan Restaurant,Historic Site,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Other Repair Shop,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio
0,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Amruthahalli,0,0,0,0,0,0

<h5> Now all venues in same neighbourhood are grouped and mean of each venue category of said venues is found </h5>

In [15]:
bgr_group = bgr_onehot.groupby('Neighbourhood').mean().reset_index()
bgr_group

,Neighbourhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Henan Restaurant,Historic Site,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Other Repair Shop,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio
0,Achitnagar,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
1,Adugodi,0.00,0.000000,0.000000,0.

<h5> Now let us find the 5 venue categories with highest mean for each neighbourhood </h5>

In [16]:
num_top_venues = 5
for hood in bgr_group['Neighbourhood']:
    print("----"+hood+"----")
    temp = bgr_group[bgr_group['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Achitnagar----
               venue  freq
0             Bakery  0.25
1  Food & Drink Shop  0.25
2         Restaurant  0.25
3   Asian Restaurant  0.25
4  Other Repair Shop  0.00


----Adugodi----
               venue  freq
0  Indian Restaurant  0.13
1             Lounge  0.07
2       Dessert Shop  0.07
3               Café  0.07
4          Multiplex  0.04


----Alahalli----
               venue  freq
0  Food & Drink Shop   1.0
1                ATM   0.0
2              Plaza   0.0
3    Motorcycle Shop   0.0
4           Mountain   0.0


----Amruthahalli----
                venue  freq
0   Indian Restaurant  0.31
1         Bus Station  0.19
2  Chinese Restaurant  0.06
3               Hotel  0.06
4        Burger Joint  0.06


----Anekal----
               venue  freq
0                ATM  0.25
1  Indian Restaurant  0.25
2       Camera Store  0.25
3   Business Service  0.25
4  Other Repair Shop  0.00


----Bagalgunte----
         venue  freq
0  Pizza Place  0.50
1   Restaurant  0.25
2   

<h5> Creating a function to extract top n popular venue categories for each neighbourhood where n is a parameter  </h5>

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h5> Using above created function to find top 10 venue categories for each neighbourhood </h5>

In [18]:

num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = bgr_group['Neighbourhood']
for ind in np.arange(bgr_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bgr_group.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Achitnagar,Food & Drink Shop,Bakery,Asian Restaurant,Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market
1,Adugodi,Indian Restaurant,Café,Dessert Shop,Lounge,Bookstore,Coffee Shop,Multiplex,Brewery,Donut Shop,Restaurant
2,Alahalli,Food & Drink Shop,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
3,Amruthahalli,Indian Restaurant,Bus Station,Ice Cream Shop,Brewery,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place,Hotel
4,Anekal,ATM,Business Service,Camera Store,Indian Restaurant,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm


<h5> Now let us perform K-means clustering on neighbourhoods based on popular venues </h5>

In [19]:
kclusters = 10

bgr_group_clustering = bgr_group.drop('Neighbourhood', 1)
bgr_group_clustering.head()

,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Henan Restaurant,Historic Site,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Other Repair Shop,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio
0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.00,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044444,0.0,0.0,0.022222,0.0,0.044444,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.00,0.0,0.0,0.066667,0.00,0.0,0.022222,0.0,0.022222,0.0,0.044444,0.0,0.022222,0.022222,0.0,0.0,0.0,0.066667,0.0,0.044444,0.022222,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.022222,0.0,0.0,0.00,0.022222,0.0,0.0,0.0,0.0,0.022222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,0.0,0.022222,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.022222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,0.0,0.044444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [20]:
bgr_group_clustering

,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Henan Restaurant,Historic Site,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Other Repair Shop,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Plaza,Toy / Game Store,Trail,Train Station,Travel & Transport,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio
0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
1,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0

 After clustering,let us view some of cluster labels returned for first 10 neighbourhoods 

In [21]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bgr_group_clustering)
kmeans.labels_[0:10]

array([0, 0, 9, 0, 0, 7, 0, 0, 0, 0])

Creating another column for cluster labels

In [22]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [23]:
afloc_df.rename(columns={'Neighborhood':'Neighbourhood'},inplace=True)
afloc_df.head()

C:\Users\Aniruddh\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Neighbourhood,Latitude,Longitude
0,Amruthahalli,13.066513,77.596624
1,Banaswadi,13.014162,77.651854
2,Bhattarahalli,13.025800,77.714279
3,Byatarayanapura,13.062074,77.596392
4,Devasandra,12.757227,77.647280


<h5> Now original dataframe on neighbourhood co-ordinates is merged with dataframe containing cluster labels based on neighbourhood names </h5>

In [25]:
bgr_merged = afloc_df

bgr_merged = bgr_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

bgr_merged.head()

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amruthahalli,13.066513,77.596624,0.0,Indian Restaurant,Bus Station,Ice Cream Shop,Brewery,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place,Hotel
1,Banaswadi,13.014162,77.651854,0.0,Indian Restaurant,Café,Pizza Place,Intersection,Bistro,Market,Department Store,Arts & Crafts Store,Vegetarian / Vegan Restaurant,Andhra Restaurant
2,Bhattarahalli,13.025800,77.714279,0.0,Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Event Service,Pizza Place,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market
3,Byatarayanapura,13.062074,77.596392,0.0,Indian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Coffee Shop,Chinese Restaurant,Café,Fast Food Restaurant,Bus Station,Burger Joint
4,Devasandra,12.757227,77.647280,8.0,Henan Restaurant,Yoga Studio,Electronics Store,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space


Cluster labels are in float and must be converted to integer.Also rows containing NaN information must be dropped

In [26]:
bgr_merged['Cluster Labels']=bgr_merged['Cluster Labels'].astype('Int64')
bgr_merged.dropna(inplace=True)

<h5> Visualizing each neighbourhood with colour on marker indicating cluster neighbourhood belongs to </h5>

In [27]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(bgr_merged['Latitude'],bgr_merged['Longitude'],bgr_merged['Neighbourhood'],bgr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

<h5> Now let us see all neighbourhoods falling under each cluster </h5>

In [28]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 0,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amruthahalli,Indian Restaurant,Bus Station,Ice Cream Shop,Brewery,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place,Hotel
1,Banaswadi,Indian Restaurant,Café,Pizza Place,Intersection,Bistro,Market,Department Store,Arts & Crafts Store,Vegetarian / Vegan Restaurant,Andhra Restaurant
2,Bhattarahalli,Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Event Service,Pizza Place,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market
3,Byatarayanapura,Indian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Coffee Shop,Chinese Restaurant,Café,Fast Food Restaurant,Bus Station,Burger Joint
5,Doddanekkundi,Coffee Shop,Indian Restaurant,Department Store,Multiplex,Movie Theater,Fried Chicken Joint,Smoke Shop,Breakfast Spot,Park,Mediterranean Restaurant
6,Domlur,Indian Restaurant,Café,Pub,Italian Restaurant,Coffee Shop,Restaurant,Hotel,Pizza Place,Diner,Lounge
7,EPIP,Hotel,Restaurant,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Salad Place,Maharashtrian Restaurant,Soccer Field,Breakfast Spot,Brewery
8,Hoodi,Indian Restaurant,Hotel,Soccer Field,Japanese Restaurant,Diner,Platform,Cocktail Bar,Bus Station,Brewery,Coffee Shop
9,Horamavu,Department Store,Badminton Court,Indian Restaurant,Andhra Restaurant,Athletics & Sports,Event Service,Food,Flea Market,Financial or Legal Service,Field
10,Indiranagar S.O (Bangalore),Indian Restaurant,Café,Lounge,Pub,Ice Cream Shop,Bar,Chinese Restaurant,Cocktail Bar,Restaurant,Bakery


In [29]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 1,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Sakalavara,Farm,Yoga Studio,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Event Space
91,Jalige,Farm,Yoga Studio,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Event Space


In [30]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 2,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Singanayakanahalli,Café,Indian Restaurant,Hotel,Electronics Store,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm
107,Uganavadi,Café,Yoga Studio,Event Service,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm


In [31]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 4,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Shivakote,Other Repair Shop,Yoga Studio,Event Service,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm


In [32]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 3,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Kalkere,Resort,Yoga Studio,Electronics Store,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space


In [33]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 5,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Taralu,ATM,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
62,Laggere,ATM,Pizza Place,Event Service,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm


In [34]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 6,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,Narasipura,Platform,Convenience Store,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space


In [35]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 7,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Hulimangala,Pizza Place,Yoga Studio,Event Service,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm
54,Bagalgunte,Pizza Place,Restaurant,Bakery,Electronics Store,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm


In [36]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 8,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Devasandra,Henan Restaurant,Yoga Studio,Electronics Store,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space


In [37]:
bgr_merged.loc[bgr_merged['Cluster Labels'] == 9,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,Alahalli,Food & Drink Shop,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space


<h5> Since most of neighbourhoods with public transport fall under cluster label 0,let us save those areas in a new dataframe </h5>

In [38]:

domtransport_df=bgr_merged.loc[bgr_merged['Cluster Labels']==0,bgr_merged.columns[[0] + list(range(4,bgr_merged.shape[1]))]]

In [39]:
domtransport_df

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amruthahalli,Indian Restaurant,Bus Station,Ice Cream Shop,Brewery,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place,Hotel
1,Banaswadi,Indian Restaurant,Café,Pizza Place,Intersection,Bistro,Market,Department Store,Arts & Crafts Store,Vegetarian / Vegan Restaurant,Andhra Restaurant
2,Bhattarahalli,Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Event Service,Pizza Place,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market
3,Byatarayanapura,Indian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Coffee Shop,Chinese Restaurant,Café,Fast Food Restaurant,Bus Station,Burger Joint
5,Doddanekkundi,Coffee Shop,Indian Restaurant,Department Store,Multiplex,Movie Theater,Fried Chicken Joint,Smoke Shop,Breakfast Spot,Park,Mediterranean Restaurant
6,Domlur,Indian Restaurant,Café,Pub,Italian Restaurant,Coffee Shop,Restaurant,Hotel,Pizza Place,Diner,Lounge
7,EPIP,Hotel,Restaurant,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Salad Place,Maharashtrian Restaurant,Soccer Field,Breakfast Spot,Brewery
8,Hoodi,Indian Restaurant,Hotel,Soccer Field,Japanese Restaurant,Diner,Platform,Cocktail Bar,Bus Station,Brewery,Coffee Shop
9,Horamavu,Department Store,Badminton Court,Indian Restaurant,Andhra Restaurant,Athletics & Sports,Event Service,Food,Flea Market,Financial or Legal Service,Field
10,Indiranagar S.O (Bangalore),Indian Restaurant,Café,Lounge,Pub,Ice Cream Shop,Bar,Chinese Restaurant,Cocktail Bar,Restaurant,Bakery


<h5> Selecting areas with public transport in top 10 popular venues within a 1 kilometre radius </h5>

In [44]:
chosen_df=domtransport_df.loc[[0,3,8,26,27,28,40,45,52,57,60,63,64,66,67,70,71]]

In [45]:
chosen_df.rename(columns={'Neighborhood':'Neighbourhood'},inplace=True)
chosen_df

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amruthahalli,Indian Restaurant,Bus Station,Ice Cream Shop,Brewery,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place,Hotel
3,Byatarayanapura,Indian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Coffee Shop,Chinese Restaurant,Café,Fast Food Restaurant,Bus Station,Burger Joint
8,Hoodi,Indian Restaurant,Hotel,Soccer Field,Japanese Restaurant,Diner,Platform,Cocktail Bar,Bus Station,Brewery,Coffee Shop
26,Chikkalasandra,Department Store,Pizza Place,Gym,Fast Food Restaurant,Sports Bar,Bus Station,Restaurant,Café,Bakery,Indian Restaurant
27,Deepanjalinagar,Indian Restaurant,Metro Station,Bus Station,Smoke Shop,Department Store,Travel & Transport,Train Station,Gas Station,Flea Market,Financial or Legal Service
28,Doddakallasandra,Bus Station,South Indian Restaurant,Indie Movie Theater,Boarding House,Arts & Entertainment,Tennis Court,Farm,Food & Drink Shop,Food,Flea Market
40,Kengeri,Bus Station,Indian Restaurant,Department Store,Café,Grocery Store,Deli / Bodega,Dessert Shop,Flea Market,Financial or Legal Service,Field
45,Nayandahalli,Metro Station,Gas Station,Train Station,Road,Electronics Store,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market
52,Yelachenahalli,Pizza Place,Metro Station,Department Store,Café,Yoga Studio,Event Service,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant
57,Hessarghatta,Pizza Place,Bakery,Gas Station,Light Rail Station,Athletics & Sports,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market


In [48]:
bmv_df= afloc_df

bmv_df = bmv_df.join(chosen_df.set_index('Neighbourhood'), on='Neighbourhood')

bmv_df.dropna(inplace=True)

<h5> Preparing dataframe for another Foursquare API call </h5>

In [49]:
cols = [3,4,5,6,7,8,9,10,11,12]
bmv_df.drop(bmv_df.columns[cols],axis=1,inplace=True)
bmv_df

,Neighbourhood,Latitude,Longitude
0,Amruthahalli,13.066513,77.596624
3,Byatarayanapura,13.062074,77.596392
8,Hoodi,12.991903,77.716201
26,Chikkalasandra,12.912192,77.547634
27,Deepanjalinagar,12.950624,77.536554
28,Doddakallasandra,12.885735,77.555745
40,Kengeri,12.917657,77.483757
45,Nayandahalli,12.945401,77.527810
52,Yelachenahalli,12.897971,77.570085
57,Hessarghatta,13.048426,77.507623


<h5> Duplicate rows are present for Mathikere so one of them is dropped </h5>

In [50]:
bmv_df.drop(98,inplace=True)
bmv_df

,Neighbourhood,Latitude,Longitude
0,Amruthahalli,13.066513,77.596624
3,Byatarayanapura,13.062074,77.596392
8,Hoodi,12.991903,77.716201
26,Chikkalasandra,12.912192,77.547634
27,Deepanjalinagar,12.950624,77.536554
28,Doddakallasandra,12.885735,77.555745
40,Kengeri,12.917657,77.483757
45,Nayandahalli,12.945401,77.527810
52,Yelachenahalli,12.897971,77.570085
57,Hessarghatta,13.048426,77.507623


<h5> Visualizing areas with public transport closeby </h5>

In [51]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lon, poi in zip(bmv_df['Latitude'],bmv_df['Longitude'],bmv_df['Neighbourhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

<h5> Creating a function for calling API similar to previous one but the radius is now 5 kilometres </h5>

In [52]:
def getCompetingVenues(names, latitudes, longitudes):
    radius=5000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h5> Using same techniques(aside from clustering) as before </h5>

In [53]:
transp_venues = getCompetingVenues(names=bmv_df['Neighbourhood'],
                                   latitudes=bmv_df['Latitude'],
                                   longitudes=bmv_df['Longitude']
                                  )

Amruthahalli
Byatarayanapura
Hoodi
Chikkalasandra
Deepanjalinagar
Doddakallasandra
Kengeri
Nayandahalli
Yelachenahalli
Hessarghatta
Kamakshipalya
Mathikere
Msrit
Nagasandra S.O (Bangalore)
Peenya Dasarahalli
Vidyaranyapura
Vijayanagar S.O (Bangalore)


In [54]:
transp_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amruthahalli,13.066513,77.596624,The Druid Garden,13.063946,77.591492,Brewery
1,Amruthahalli,13.066513,77.596624,Big Straw,13.063414,77.591192,Bubble Tea Shop
2,Amruthahalli,13.066513,77.596624,Jus'Trufs Chocolate Shop and Cafe,13.072510,77.603859,Café
3,Amruthahalli,13.066513,77.596624,Swensen's,13.063476,77.590793,Ice Cream Shop
4,Amruthahalli,13.066513,77.596624,Godrej Woodsman Estate,13.054715,77.595815,Building


In [55]:
transp_onehot = pd.get_dummies(transp_venues[['Venue Category']], prefix="", prefix_sep="")
transp_onehot['Neighbourhood'] = transp_venues['Neighborhood'] 
fixed_col = [transp_onehot.columns[-1]] + list(transp_onehot.columns[:-1])
transp_onehot = transp_onehot[fixed_col]

transp_onehot.head()

,Neighbourhood,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Garden,Gas Station,General Entertainment,Gym,Gym / Fitness Center,Gym Pool,Hotel,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Liquor Store,Lounge,Maharashtrian Restaurant,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Neighborhood,Nightclub,Office,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Amruthahalli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
transp_group = transp_onehot.groupby('Neighbourhood').mean().reset_index()
transp_group

,Neighbourhood,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Garden,Gas Station,General Entertainment,Gym,Gym / Fitness Center,Gym Pool,Hotel,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Liquor Store,Lounge,Maharashtrian Restaurant,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Neighborhood,Nightclub,Office,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Amruthahalli,0.010753,0.010753,0.00,0.000000,0.00,0.00,0.00,0.010753,0.010753,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.010753,0.010753,0.010753,0.010753,0.010753,0.010753,0.086022,0.021505,0.010753,0.021505,0.00,0.000000,0.00,0.00,0.021505,0.010753,0.00,0.00,0.00,0.00,0.00,0.00,0.075269,0.010753,0.000000,0.010753,0.00,0.000000,0.010753,0.00,0.00,0.021505,0.00,0.00,0.000000,0.00,0.086022,0.000000,0.215054,0.00,0.021505,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.010753,0.00,0.010753,0.010753,0.00,0.000000,0.00,0.000000,0.010753,0.00,0.086022,0.000000,0.00,0.00,0.00,0.000000,0.00,0.021505,0.000000,0.00,0.00,0.010753,0.000000,0.00,0.000000,0.010753,0.021505,0.010753,0.010753,0.00,0.000000,0.00,0.010753,0.00,0.00,0.00,0.00,0.00,0.00,0.010753,0.010753,0.021505,0.00
1,Byatarayanapura,0.010000,0.010000,0.01,0.000000,0.00,0.00,0.00,0.010000,0.010000,0.01,0.000000,0.000000,0.01,0.00,0.00,0.00,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.090000,0.040000,0.010000,0.030000,0.00,0.000000,0.00,0.00,0.020000,0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.080000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.100000,0.000000,0.150000,0.00,0.020000,0.00,0.01,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.010000,0.00,0.010000,0.010000,0.00,0.000000,0.00,0.000000,0.010000,0.00,0.100000,0.000000,0.00,0.00,0.01,0.000000,0.00,0.030000,0.000000,0.00,0.00,0.030000,0.000000,0.00,0.000000,0.010000,0.020000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.010000,0.010000,0.00
2,Chikkalasandra,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.01,0.01,0.01,0.00,0.050000,0.020000,0.000000,0.000000,0.020000,0.000000,0.050000,0.020000,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.010000,0.000000,0.01,0.00,0.00,0.00,0.00,0.01,0.050000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.01,0.00,0.000000,0.01,0.070000,0.020000,0.200000,0.00,0.020000,0.01,0.02,0.000000,0.00,0.02,0.00,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.00,0.000000,0.010000,0.01,0.040000,0.000000,0.00,0.00,0.00,0.000000,0.01,0.000000,0.030000,0.01,0.00,0.040000,0.010000,0.01,0.000000,0.000000,0.040000,0.000000,0.030000,0.01,0.000000,0.00,0.000000,0.01,0.01,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.01
3,Deepanjalinagar,0.000000,0

In [57]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
best_venues_sorted = pd.DataFrame(columns=columns)
best_venues_sorted['Neighbourhood'] = transp_group['Neighbourhood']
for ind in np.arange(transp_group.shape[0]):
    best_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bgr_group.iloc[ind, :], num_top_venues)
best_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amruthahalli,Food & Drink Shop,Bakery,Asian Restaurant,Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market
1,Byatarayanapura,Indian Restaurant,Café,Dessert Shop,Lounge,Bookstore,Coffee Shop,Multiplex,Brewery,Donut Shop,Restaurant
2,Chikkalasandra,Food & Drink Shop,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
3,Deepanjalinagar,Indian Restaurant,Bus Station,Ice Cream Shop,Brewery,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place,Hotel
4,Doddakallasandra,ATM,Business Service,Camera Store,Indian Restaurant,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm


In [58]:
best_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amruthahalli,Food & Drink Shop,Bakery,Asian Restaurant,Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market
1,Byatarayanapura,Indian Restaurant,Café,Dessert Shop,Lounge,Bookstore,Coffee Shop,Multiplex,Brewery,Donut Shop,Restaurant
2,Chikkalasandra,Food & Drink Shop,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
3,Deepanjalinagar,Indian Restaurant,Bus Station,Ice Cream Shop,Brewery,Chinese Restaurant,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Pizza Place,Hotel
4,Doddakallasandra,ATM,Business Service,Camera Store,Indian Restaurant,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
5,Hessarghatta,Pizza Place,Restaurant,Bakery,Electronics Store,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm
6,Hoodi,Food Truck,Memorial Site,Yoga Studio,Event Space,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market
7,Kamakshipalya,Indian Restaurant,Café,Pizza Place,Intersection,Bistro,Market,Department Store,Arts & Crafts Store,Vegetarian / Vegan Restaurant,Andhra Restaurant
8,Kengeri,Indian Restaurant,Fast Food Restaurant,Ice Cream Shop,Café,Pizza Place,Department Store,Gym,Gastropub,Hotel Bar,Electronics Store
9,Mathikere,Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Event Service,Pizza Place,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market


<h5> Dropping all neighbourhoods with no footfall(observed from areas like flea markets) and competing venues popular in malls like non-local cuisine restaurants and theatres to get final list of best neighbourhoods to build a mall </h5>

In [59]:
th_lst=[0,1,3,5,6,7,8,9,11,12,14]
best_venues_sorted.drop(th_lst,inplace=True)

In [60]:
final_df=best_venues_sorted
final_df

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Chikkalasandra,Food & Drink Shop,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
4,Doddakallasandra,ATM,Business Service,Camera Store,Indian Restaurant,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
10,Msrit,Snack Place,Yoga Studio,Food & Drink Shop,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
13,Peenya Dasarahalli,Hotel,Bike Shop,Fast Food Restaurant,Café,Yoga Studio,Event Service,Flea Market,Financial or Legal Service,Field,Farmers Market
15,Vijayanagar S.O (Bangalore),Indian Restaurant,Metro Station,Bus Station,Smoke Shop,Department Store,Travel & Transport,Train Station,Gas Station,Flea Market,Financial or Legal Service
16,Yelachenahalli,Indian Restaurant,Andhra Restaurant,Electronics Store,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm


In [61]:
mv_df= afloc_df

mv_df = mv_df.join(final_df.set_index('Neighbourhood'), on='Neighbourhood')

mv_df.dropna(inplace=True)

In [62]:
mv_df

,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Chikkalasandra,12.912192,77.547634,Food & Drink Shop,Indian Restaurant,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
28,Doddakallasandra,12.885735,77.555745,ATM,Business Service,Camera Store,Indian Restaurant,Hotel,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
52,Yelachenahalli,12.897971,77.570085,Indian Restaurant,Andhra Restaurant,Electronics Store,Food,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm
64,Msrit,13.030955,77.564856,Snack Place,Yoga Studio,Food & Drink Shop,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant,Farmers Market,Farm,Event Space
67,Peenya Dasarahalli,13.033019,77.533201,Hotel,Bike Shop,Fast Food Restaurant,Café,Yoga Studio,Event Service,Flea Market,Financial or Legal Service,Field,Farmers Market
71,Vijayanagar S.O (Bangalore),12.971889,77.545789,Indian Restaurant,Metro Station,Bus Station,Smoke Shop,Department Store,Travel & Transport,Train Station,Gas Station,Flea Market,Financial or Legal Service


<h5> Visualizing all areas which are suitable to build malls in </h5>

In [63]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lon, poi in zip(mv_df['Latitude'],mv_df['Longitude'],mv_df['Neighbourhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
map_clusters